In [69]:
import requests
from bs4 import BeautifulSoup as bs
import time
import csv

def get_car_links(main_url, page_number):
    try:
        url = f"{main_url}?o={page_number}"
        response = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
        response.raise_for_status()

        soup = bs(response.text, 'html.parser')
        car_links = [a['href'] for a in soup.select('a.olx-ad-card__link-wrapper')]
        return car_links
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des liens de la page {page_number} : {e}")
        return []

def scrape_car_details(car_url):
    try:
        response = requests.get(car_url, headers={'User-agent': 'Mozilla/5.0'})
        response.raise_for_status()

        car_soup = bs(response.text, 'html.parser')

        # Obtenez les détails de la voiture
        car_details = car_soup.find_all(class_='ad__sc-1g2w54p-1 WNDyV olx-d-flex olx-fd-column')

        # Obtenez le prix de la voiture
        car_price_element = car_soup.find(class_='ad__sc-12l420o-0 fBIVnd olx-d-flex olx-ai-center olx-fd-row')
        car_price = car_price_element.text.strip() if car_price_element else "Prix non trouvé"

        # Processus des données récupérées
        details_text = '\n'.join(detail.text for detail in car_details)
        print(details_text)
        print(f"Prix de la voiture: {car_price}\n{'='*30}")

        return {'Car Details': details_text, 'Car Price': car_price}

    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des détails de la voiture : {e}")
        return None

main_page_url = 'https://www.olx.com.br/autos-e-pecas/carros-vans-e-utilitarios'
total_pages = 110  # Modifier le nombre total de pages selon les besoins
car_details_list = []

for page_number in range(1, total_pages + 1):
    car_links = get_car_links(main_page_url, page_number)

    for car_link in car_links:
        print(f"Scrapping details for car: {car_link}")
        car_detail = scrape_car_details(car_link)
        
        if car_detail:
            car_details_list.append(car_detail)
        else:
            print(f"Aucun détail trouvé pour la voiture : {car_link}")
        
        time.sleep(2)  # Respectez les politiques du site pour éviter d'être bloqué

# Ajout de débogage
print("Nombre total de détails récupérés :", len(car_details_list))

# Écrire les détails des voitures dans un fichier CSV
with open('car_details.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Car Details', 'Car Price']  # Ajoutez 'Car Price' à la liste
    csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    csv_writer.writeheader()
    
    for detail in car_details_list:
        csv_writer.writerow(detail)


Scrapping details for car: https://sc.olx.com.br/florianopolis-e-regiao/autos-e-pecas/carros-vans-e-utilitarios/jeep-commander-2022-1256411838
CaracterísticasJEEP COMMANDER OVERL. TD380 2.0 4X4 DIE. AUTCategoriaCarros, vans e utilitáriosModeloJEEP COMMANDER OVERL. TD380 2.0 4X4 DIE. AUTMarcaJEEPTipo de veículoSUVAno2022Quilometragem8000CombustívelDieselPossui Kit GNVNãoCâmbioAutomáticoCorPretoFinal de placa3Tipo de direçãoElétricaVer OpcionaisOpcionaisAlarmeAir bagAr condicionadoCâmera de réSensor de réSomTrava elétricaVidro elétrico
Prix de la voiture: R$230.000
Scrapping details for car: https://sp.olx.com.br/grande-campinas/autos-e-pecas/carros-vans-e-utilitarios/honda-civic-touring-turbo-1234568472
CaracterísticasHONDA CIVIC SEDAN TOURING 1.5 TURBO 16V AUT.4PCategoriaCarros, vans e utilitáriosModeloHONDA CIVIC SEDAN TOURING 1.5 TURBO 16V AUT.4PMarcaHONDATipo de veículoSedãAno2019Quilometragem88000Potência do motor1.5CombustívelGasolinaPossui Kit GNVNãoCâmbioAutomáticoCorBrancoPorta

In [71]:
import requests
from bs4 import BeautifulSoup as bs
import time
import csv
from urllib.parse import urljoin

def get_car_links(main_url, page_number):
    try:
        url = f"{main_url}&o={page_number}"
        response = requests.get(url, headers={'User-agent': 'Mozilla/5.0'})
        response.raise_for_status()

        soup = bs(response.text, 'html.parser')
        base_url = "https://www.icarros.com.br"  # Ajoutez le domaine de base ici
        car_links = [urljoin(base_url, a['href']) for a in soup.select('a.offer-card__title-container')]
        return car_links
    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des liens de la page {page_number} : {e}")
        return []

def scrape_car_details(car_url):
    try:
        response = requests.get(car_url, headers={'User-agent': 'Mozilla/5.0'})
        response.raise_for_status()

        car_soup = bs(response.text, 'html.parser')

        # Obtenez le nom de la voiture
        car_name_element = car_soup.select_one('.titulo-sm')
        car_name = car_name_element.get_text(strip=True) if car_name_element else "Nom non trouvé"

        # Obtenez les détails de la voiture
        car_details_elements = car_soup.select('.card-conteudo')
        car_features = {}

        for detail_element in car_details_elements:
            feature_icon = detail_element.select_one('.icones')
            feature_name = detail_element.select_one('p').get_text(strip=True) if detail_element.select_one('p') else "Nom de la caractéristique non trouvé"

            if feature_icon:
                feature_class = feature_icon.get('class')[1]
                car_features[feature_class] = feature_name

        # Ajoutez du code pour extraire le prix de la voiture
        car_price_element = car_soup.select_one('.preco')
        car_price = car_price_element.get_text(strip=True) if car_price_element else "Prix non trouvé"

        print(f"Nom de la voiture: {car_name}")
        print(f"Caractéristiques de la voiture:\n{car_features}")
        print(f"Prix de la voiture: {car_price}\n{'='*30}")

        return {'Car Name': car_name, 'Car Features': car_features, 'Car Price': car_price}

    except requests.exceptions.RequestException as e:
        print(f"Erreur lors de la récupération des détails de la voiture : {e}")
        return None


main_page_url = 'https://www.icarros.com.br/comprar/usados?reg=city'
total_pages = 1400  # Modifier le nombre total de pages selon les besoins
car_details_list = []

for page_number in range(1, total_pages + 1):
    car_links = get_car_links(main_page_url, page_number)

    for car_link in car_links:
        print(f"Scrapping details for car: {car_link}")
        car_detail = scrape_car_details(car_link)
        
        if car_detail:
            car_details_list.append(car_detail)
        else:
            print(f"Aucun détail trouvé pour la voiture : {car_link}")
        
        time.sleep(2)  # Respectez les politiques du site pour éviter d'être bloqué

# Ajout de débogage
print("Nombre total de détails récupérés :", len(car_details_list))

# Écrire les détails des voitures dans un fichier CSV
with open('car_details_icarros.csv', 'w', newline='', encoding='utf-8') as csvfile:
    fieldnames = ['Car Name', 'Car Features', 'Car Price']  # Ajoutez 'Car Price' à la liste
    csv_writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    csv_writer.writeheader()
    
    for detail in car_details_list:
        csv_writer.writerow(detail)


Scrapping details for car: https://www.icarros.com.br/comprar/ponta-grossa-pr/fiat/mobi/2021/d44677974?pos=1&hfv=false&financiamento=true
Nom de la voiture: Fiat Mobi1.0 Like 2021
Caractéristiques de la voiture:
{'icone-interrogacao': 'preço à vista', 'icones_opcionais': 'Flex, \n                                    \n                                    \n                                        Final da placa 7'}
Prix de la voiture: R$ 47.990,00
Scrapping details for car: https://www.icarros.com.br/comprar/balneario-camboriu-sc/fiat/uno/2021/d44682264?pos=2&hfv=false&financiamento=true
Nom de la voiture: Fiat Uno1.0 Attractive 2021
Caractéristiques de la voiture:
{'icone-interrogacao': 'preço à vista', 'icones_opcionais': 'Flex, \n                                    \n                                    \n                                        Final da placa 2'}
Prix de la voiture: R$ 47.990,00
Scrapping details for car: https://www.icarros.com.br/comprar/balneario-camboriu-sc/fiat/uno

KeyboardInterrupt: 